In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from ipwhois import IPWhois

### Get topology from Ripple

In [2]:
url = 'https://data.ripple.com/v2/network/topology?verbose=True'
result = requests.get(url).json()

### Extracting nodes and links dataframes

In [3]:
nodes_raw = result['nodes']
links_raw = result['links']

nodes = pd.DataFrame.from_dict(nodes_raw)
links = pd.DataFrame.from_dict(links_raw)

# Observations

## Geographic 

### What are the most important cities?

In [4]:
top_cities = nodes[['city','lat','long']].copy()
city_count = top_cities.groupby('city').size()

top_cities = top_cities.dropna().drop_duplicates()
top_cities = top_cities.reset_index().set_index('city')
top_cities['city_count'] = city_count
top_cities.sort_values(by='city_count', ascending=False).head(10)

,index,lat,long,city_count
city,,,,
Ann Arbor,86,42.2734,-83.7133,63
Cambridge,101,42.3646,-71.1028,49
Ottawa,71,45.3433,-75.8265,49
Cambridge,507,42.38,-71.1329,49
Houston,9,29.6997,-95.5858,40
Ashburn,1,39.018,-77.539,32
Boardman,3,45.8696,-119.688,29
Tokyo,82,35.6845,139.7559,29
Singapore,83,1.2931,103.8558,24


### What are the most important countries?

In [5]:
nodes.groupby('country').size().sort_values(ascending=False).reset_index().head(10)

,country,0
0,United States,500
1,Germany,81
2,Canada,78
3,"Korea, Republic of",63
4,China,49
5,Japan,47
6,Ukraine,29
7,Singapore,24
8,United Kingdom,20
9,Russian Federation,20


In [6]:
links['source_city'] = links['source'].map(nodes.set_index('node_public_key')['city'])
links['target_city'] = links['target'].map(nodes.set_index('node_public_key')['city'])
links['source_country'] = links['source'].map(nodes.set_index('node_public_key')['country'])
links['target_country'] = links['target'].map(nodes.set_index('node_public_key')['country'])

In [7]:
links_city = links.copy().dropna()

### Which cities are the most connected?

In [8]:
links_city.groupby(['source_city', 'target_city']).size().sort_values(ascending=False).reset_index().head(10)

,source_city,target_city,0
0,Boardman,Boardman,224
1,Boardman,Houston,183
2,Boardman,Ann Arbor,181
3,Boardman,Ashburn,151
4,Boardman,Cambridge,136
5,Ashburn,Houston,133
6,Houston,Boardman,126
7,Houston,Ann Arbor,122
8,Houston,Houston,121
9,Boardman,Tokyo,114


### Which cities send money?

In [9]:
links_city.groupby('source_city').size().sort_values(ascending=False).reset_index().head(10)

,source_city,0
0,Boardman,2064
1,Houston,1248
2,Ashburn,1093
3,Dallas,659
4,San Jose,239
5,London,212
6,Ann Arbor,192
7,Chantilly,174
8,Incheon,148
9,Kansas City,66


### Which cities receive money?

In [10]:
links_city.groupby('target_city').size().sort_values(ascending=False).reset_index().head(10)

,target_city,0
0,Boardman,777
1,Houston,694
2,Ann Arbor,620
3,Ashburn,613
4,Cambridge,406
5,Ottawa,340
6,Dallas,311
7,Tokyo,224
8,San Jose,210
9,Singapore,204


### Which countries are the most connected?

In [11]:
links_city.groupby(['source_country', 'target_country']).size().sort_values(ascending=False).reset_index().head(10)

,source_country,target_country,0
0,United States,United States,4744
1,United States,Canada,460
2,United States,Japan,200
3,United States,Germany,189
4,United States,Singapore,181
5,United Kingdom,United States,138
6,Canada,United States,111
7,United States,"Korea, Republic of",110
8,United States,Netherlands,88
9,"Korea, Republic of",United States,85


### Which countries send money?

In [12]:
links_city.groupby('source_country').size().sort_values(ascending=False).reset_index().head(10)

,source_country,0
0,United States,6355
1,United Kingdom,219
2,Canada,155
3,"Korea, Republic of",148
4,Germany,57
5,Netherlands,43
6,Ireland,29
7,Singapore,27
8,Japan,19
9,Iceland,12


### Which countries receive money?

In [13]:
links_city.groupby('target_country').size().sort_values(ascending=False).reset_index().head(10)

,target_country,0
0,United States,5239
1,Canada,541
2,Germany,229
3,Japan,224
4,Singapore,204
5,"Korea, Republic of",121
6,Netherlands,100
7,Ireland,83
8,United Kingdom,80
9,Brazil,66


## Get ASN from IP 

In [28]:
def get_as(x):
    try:
        obj = IPWhois(x)
        result = obj.lookup_whois()
        return  result['asn']
    except:
        print('ERROR with ' + str(x))
        return np.nan

In [29]:
nodes['asn'] = nodes['ip'].apply(lambda x: get_as(x))

ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with ::ffff:139.162.114.19
ERROR with ::ffff:94.237.37.114
ERROR with ::ffff:95.216.77.89
ERROR with ::ffff:109.70.144.110
ERROR with ::ffff:197.237.170.19
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with ::ffff:5.35.241.18
ERROR with ::ffff:68.183.90.147
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with 172.17.0.1
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan
ERROR with nan


In [31]:
nodes[['node_public_key','ip','asn']]

,node_public_key,ip,asn
0,n9KDJqdL5pebR35tkE4pRuZ7MhTvQGid8YqwjJajwrZUN5...,169.44.60.105,36351
1,n9KJb7NMxGySRcjCqh69xEPMUhwJx22qntYYXsnUqYgjsJ...,54.84.21.230,14618
2,n9KEUBck9R94pBrebAK1d6NvGJAeDJzeHL14QX89MsFVKm...,54.172.76.54,14618
3,n9LbivWTCipEhgQBSBnCMyfHNvCiAbpB9untffzTALPYwY...,54.213.156.82,16509
4,n9MEp68R56NjLaNgSpia8SafZoALMsq9F4bxbfZ68N8pcE...,54.160.239.185,14618
5,n9Jk1NL9wCFQtrKYURNWpWQyJ5xEJ3SVrKhaj5skZhwTrv...,NaN,NaN
6,n9LkwyXNWRjYv7sR1Ks9D8XbEhabeNPH9nCNViYmV7ouFC...,NaN,NaN
7,n9LnqwvUxFBUdnS1JzbziohwVium9EkYL5i7TuTx2TvPSs...,169.54.2.157,36351
8,n94p4za2joNkhwsU9cvAEmHM97LKfQEDzDzFBUzAYryLUv...,169.55.164.29,36351
9,n9KiJ4JHhajZ3mQ2k29ekz1ixnDYaQdF6bNi2hiKDhRM9n...,34.239.120.20,14618
